<a href="https://colab.research.google.com/github/satishgunjal/Machine-Learning-Using-Python/blob/master/14_Naive_Bayes/Naive_Bayes_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive bayes Algorithm Part 2

## Problem Statement
* Spam email classification

## Count Vectorizer Technique
* We are going to use count vectorizer technique for converting the mail body into numbers.
* How it works: We will identify the unique words accorss all the mails and then find the count of each word in every mail. So we get the matrix with columns equal to no of unique words and rows equal to number of mails and each cell represent the count of that unique word(in that mail)

## Load the spam data

In [0]:
import pandas as pd

In [0]:
#For local Notebook
#df = pd.read_csv('spam.csv')
#df.head()

In [2]:
# I am using google Colaboratory notebook
from google.colab import files

uploaded = files.upload()

Saving spam.csv to spam.csv


In [9]:
import io

df = pd.read_csv(io.StringIO(uploaded['spam.csv'].decode('utf-8')))
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.groupby('Category').describe()

Message                                                               
           count unique                                                top freq
Category                                                                       
ham         4825   4516                             Sorry, I'll call later   30
spam         747    641  Please call our customer service representativ...    4

## Understanding the dataset
* Since all the data is in text format we must convrt it into numeric for our ML algorithm to work
* We can use One Hot Encode for column'Category'
* Since 'message' column data is not categorical we can use 'Count Vectorizer' to convert the text into numeric format

Since Category column has only two categories(spam and ham) we can also assign lebel using lambda function

In [12]:
# Create new column 'spam' where 1 is for spam and 0 for ham
df['spam'] = df.Category.apply(lambda x : 1 if x == 'spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


Now lets split the dataset in train and test

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(df.Message,df.spam,test_size=0.25)

print("len of X_train is %s" % (len(X_train)))
print("len of X_test is %s" % (len(X_test)))
print("len of y_train is %s" % (len(y_train)))
print("len of y_test is %s" % (len(y_test)))

len of X_train is 4179
len of X_test is 1393
len of y_train is 4179
len of y_test is 1393


Now lets use sklearn's 'Count Vectorizer' to convert 'Messages' column in numeric

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

v= CountVectorizer()
X_train_count = v.fit_transform(X_train.values)
X_train_count.toarray()[:2]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Every feature in above matrix represent the no of unique words in Messages column

## Lets train the model using Multinomial Naive Byaes Classifier

In [38]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [39]:
X_test_count = v.transform(X_test)
model.score(X_test_count, y_test)

0.9863603732950467

## Predict the result
* We will use two email's to test the prediction accuracy
* First email is ham and second email is spam

In [40]:
emails = [
    'Hey man, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

array([0, 1])

## Sklearn Pipeline
* As of now we were using the 'Count Vectorizer' transformer to convert the data to numeric format before using it in a model
* Using Sklearn pipeline we can simplyfy these stpes

In [0]:
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [43]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('nb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [44]:
clf.score(X_test, y_test)

0.9863603732950467

In [45]:
clf.predict(emails)

array([0, 1])